In [2]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import os
import re

# 定义文件路径和日期范围
file_dir = '../data/raw/'
file_patterns = {
    'gst': r'Z_NAFP_C_BABJ_\d+_P_CLDAS_NRT_ASI_0P0625_DAY-GST-(\d{8})\d{2}.nc',
}
district_name = '延安'
lat_min, lat_max = 35.34, 37.3   # 纬度范围
lon_min, lon_max = 108.51, 110.41  # 经度范围

# 初始化一个空的 DataFrame 用于汇总平均值
summary_df = pd.DataFrame(columns=['id', 'district_name', 'date', 'file_suffix', 'type', 'value'])

for file_suffix, pattern in file_patterns.items():
    # 初始化一个空的 DataFrame
    df = pd.DataFrame(columns=['id', 'district_name', 'date', 'type', 'value'])

    # 遍历文件夹中的所有 NetCDF 文件
    for file_name in os.listdir(file_dir):
        match = re.search(pattern, file_name)
        if match:
            date_str = match.group(1)
            date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            file_path = os.path.join(file_dir, file_name)

            # 打开 NetCDF 文件
            dataset = nc.Dataset(file_path)

            # 获取地表平均温度、经度和纬度数据
            ground_temperature_avg_day_sum = dataset.variables['TG_DAY_AVG'][:]
            latitudes = dataset.variables['LAT'][:]          # 纬度变量
            longitudes = dataset.variables['LON'][:]         # 经度变量

            # 找到符合经纬度范围的索引
            lat_indices = np.where((latitudes >= lat_min) & (latitudes <= lat_max))[0]
            lon_indices = np.where((longitudes >= lon_min) & (longitudes <= lon_max))[0]

            # 提取指定区域的数据
            ground_temperature_avg_day_sum_region = ground_temperature_avg_day_sum[np.ix_(lat_indices, lon_indices)]

            # 计算平均值（排除极端值），一般地表温度在 200 K（-73°C）到 350 K（77°C） 之间是合理的范围。
            # k 开尔文温度，绝对温度
            ground_temperature_day_avg = np.mean(ground_temperature_avg_day_sum_region[np.logical_and(ground_temperature_avg_day_sum_region >= 200, ground_temperature_avg_day_sum_region < 350)])

            # 关闭数据集
            dataset.close()

            # 创建一个新的 DataFrame 行
            new_row = pd.DataFrame({
                'id': [len(df) + 1],
                'district_name': [district_name],
                'date': [date],
                'type': ['ground_temperature_day_avg'],
                'value': [ground_temperature_day_avg]
            })

            # 使用 pd.concat 添加新行到 DataFrame
            df = pd.concat([df, new_row], ignore_index=True)

            # 创建一个新的 DataFrame 行用于汇总
            summary_row = pd.DataFrame({
                'id': [len(summary_df) + 1],
                'district_name': [district_name],
                'date': [date],
                'file_suffix': [file_suffix],
                'type': ['ground_temperature_day_avg'],
                'value': [ground_temperature_day_avg]
            })

            # 使用 pd.concat 添加新行到汇总 DataFrame
            summary_df = pd.concat([summary_df, summary_row], ignore_index=True)

    # 保存每个深度的 DataFrame 到 CSV 文件
    output_dir = '../data/csv/'
    os.makedirs(output_dir, exist_ok=True)
    df.to_csv(os.path.join(output_dir, f'ground_temperature_day_avg_data.csv'), index=False)

C:\Users\zhout\AppData\Local\Temp\ipykernel_31188\1841596321.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)
C:\Users\zhout\AppData\Local\Temp\ipykernel_31188\1841596321.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary_df = pd.concat([summary_df, summary_row], ignore_index=True)
